In [44]:
import os

In [45]:
os.chdir("E:\\datascienceproject")
%pwd

'E:\\datascienceproject'

In [46]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [47]:
from src.datascienece.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.datascienece.utils.common import read_yaml, create_directories

In [48]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH,
                 schema_file_path = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config
        

In [49]:
import os 
import urllib.request as request
from src.datascienece import logger
import zipfile

In [50]:
## component-Data ingestion 

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    # Downloading the data from source URL to local file
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
              url=self.config.source_URL, filename=self.config.local_data_file
              )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {os.path.getsize(self.config.local_data_file)} bytes")
    def extract_zip_file(self):
      
      """
      zip_file_path : str
      Extracts the zip file into the data directory
      function returns None 
      """
      unzip_dir = self.config.unzip_dir
      os.makedirs(unzip_dir, exist_ok=True)
      with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
          zip_ref.extractall(unzip_dir)
        

In [51]:
try:
  config=ConfigurationManager()
  data_ingestion_config = config.get_data_ingestion_config()
  data_ingestion = DataIngestion(config=data_ingestion_config)
  data_ingestion.download_data()
  data_ingestion.extract_zip_file()
except Exception as e:
  raise e
  

[2026-02-22 20:13:29,166: INFO: common]: yaml file: config\config.yaml loaded successfully
[2026-02-22 20:13:29,168: INFO: common]: yaml file: parameters.yaml loaded successfully
[2026-02-22 20:13:29,170: INFO: common]: yaml file: schema.yaml loaded successfully
[2026-02-22 20:13:29,171: INFO: common]: Directory created at: artifacts
[2026-02-22 20:13:29,172: INFO: common]: Directory created at: artifacts/data_ingestion
[2026-02-22 20:13:29,173: INFO: 2366901223]: File already exists of size: 6038 bytes
